# Clase27: Deep Learning cn TensorFlow

### Diferencia entre TensorFlow y Keras

Por ejemplo en este caso: [link](https://playground.tensorflow.org/#activation=tanh&batchSize=10&dataset=spiral&regDataset=reg-plane&learningRate=0.03&regularizationRate=0&noise=0&networkShape=4,2,2,2&seed=0.32536&showTestData=false&discretize=false&percTrainData=50&x=true&y=true&xTimesY=true&xSquared=true&ySquared=true&cosX=false&sinX=true&cosY=false&sinY=true&collectStats=false&problem=classification&initZero=false&hideText=false)

#### TensorFlow Puro

En TensorFlow sin Keras, tendrías que manejar más detalles de bajo nivel como la creación de variables, la construcción del gráfico computacional, y el manejo de sesiones para ejecutar los cálculos. Aquí un ejemplo de cómo podría ser:



In [ ]:
import tensorflow as tf
import numpy as np

# Definir los datos de entrada y las etiquetas
X = tf.placeholder(tf.float32, shape=[None, 2])
y = tf.placeholder(tf.float32, shape=[None, 2])

# Definir pesos y sesgos manualmente para cada capa
def dense_layer(X, input_size, output_size, activation=None):
    W = tf.Variable(tf.random_normal([input_size, output_size]), name="weights")
    b = tf.Variable(tf.zeros([output_size]), name="bias")
    z = tf.add(tf.matmul(X, W), b)
    if activation:
        z = activation(z)
    return z

# Crear la red neuronal con las capas ocultas
layer_1 = dense_layer(X, 2, 4, activation=tf.nn.tanh)  # Primera capa oculta
layer_2 = dense_layer(layer_1, 4, 2, activation=tf.nn.tanh)  # Segunda capa oculta
layer_3 = dense_layer(layer_2, 2, 2, activation=tf.nn.tanh)  # Tercera capa oculta
output = dense_layer(layer_3, 2, 2, activation=tf.nn.softmax)  # Capa de salida

# Definir el coste y el optimizador
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output))
optimizer = tf.train.AdamOptimizer(learning_rate=0.03).minimize(loss)

# Inicializar variables y entrenar el modelo
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(1000):
        # Alimentar los datos y entrenar aquí
        # sess.run(optimizer, feed_dict={X: batch_X, y: batch_y})
        pass


#### Keras

En Keras, todo este proceso se simplifica enormemente, ya que Keras maneja las capas y los detalles de la optimización por ti. El mismo modelo se puede definir y entrenar de forma mucho más intuitiva:



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Crear el modelo secuencial
model = Sequential()

# Añadir capas ocultas
model.add(Dense(4, input_dim=2, activation='tanh'))  # Primera capa oculta
model.add(Dense(2, activation='tanh'))  # Segunda capa oculta
model.add(Dense(2, activation='tanh'))  # Tercera capa oculta

# Capa de salida
model.add(Dense(2, activation='softmax'))  # Capa de salida

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.03), loss='categorical_crossentropy')

# Entrenar el modelo
# model.fit(X_train, y_train, epochs=1000, batch_size=10)


# Ejercicio: Detección de Malaria usando TensorFlow y Keras
Este ejercicio consiste en entrenar un modelo de clasificación de imágenes para detectar células infectadas con malaria.

### Paso 1: Importar las librerías necesarias
Primero, debemos importar las librerías requeridas.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

### Paso 2: Cargar el dataset de malaria
Usaremos tensorflow_datasets para cargar el conjunto de datos de malaria. Vamos a dividir el conjunto de datos en entrenamiento y validación.

In [ ]:
# Cargar el dataset de malaria
(train_dataset, test_dataset), info = tfds.load('malaria', 
                                                split=['train[:80%]', 'train[80%:]'], 
                                                as_supervised=True, 
                                                with_info=True)

# Información básica sobre el dataset
print(info)

### Paso 3: Preprocesar los datos
Es importante normalizar las imágenes, ya que los modelos de redes neuronales se entrenan mejor con valores entre 0 y 1. Normalizaremos las imágenes dividiendo los valores de los píxeles por 255.0.

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalización
    return image, label

# Aplicar la preprocesamiento a los datos
train_dataset = train_dataset.map(preprocess).batch(32).shuffle(buffer_size=1000)
test_dataset = test_dataset.map(preprocess).batch(32)

### Paso 4: Crear el modelo de red neuronal convolucional (CNN)
Usaremos una red neuronal convolucional simple con algunas capas de convolución y pooling, seguida de capas densas para la clasificación.

In [ ]:
# Crear un modelo secuencial
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(145, 145, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    
    Dense(128, activation='relu'),
    Dropout(0.5),  # Para reducir sobreajuste
    
    Dense(1, activation='sigmoid')  # Clasificación binaria
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Resumen del modelo
model.summary()

### Paso 5: Entrenar el modelo
Utilizaremos EarlyStopping para detener el entrenamiento si el modelo deja de mejorar en el conjunto de validación.

In [ ]:
# Definir un callback para early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Entrenar el modelo
history = model.fit(train_dataset, 
                    epochs=10, 
                    validation_data=test_dataset, 
                    callbacks=[early_stopping])


### Paso 6: Evaluar el modelo
Después del entrenamiento, evaluamos el rendimiento del modelo en el conjunto de datos de prueba.

In [ ]:
# Evaluar el modelo en el conjunto de test
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


### Paso 7: Visualización de resultados
Es útil visualizar las curvas de pérdida y precisión durante el entrenamiento.



In [ ]:
# Visualizar los resultados
plt.plot(history.history['accuracy'], label='Precisión en Entrenamiento')
plt.plot(history.history['val_accuracy'], label='Precisión en Validación')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Pérdida en Entrenamiento')
plt.plot(history.history['val_loss'], label='Pérdida en Validación')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

### Paso 8: Hacer predicciones
Finalmente, puedes usar el modelo para hacer predicciones en nuevas imágenes o en algunas del conjunto de prueba.


In [ ]:
# Obtener algunas imágenes del conjunto de test para predecir
for image, label in test_dataset.take(1):
    pred = model.predict(image)
    print("Predicción:", pred)
    print("Etiqueta real:", label.numpy())
